In [1]:
import pandas as pd
import numpy as np
import librosa
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Input,
    Conv1D,
    LSTM,
    Bidirectional,
    Dropout,
    BatchNormalization,
    Dense,
    MaxPooling1D,
    concatenate,
    GlobalAveragePooling1D,
    Flatten,
)

from tqdm.notebook import tqdm
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import save_model, load_model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
#from catboost import CatBoostClassifier

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, recall_score, precision_score, confusion_matrix, roc_curve

import seaborn as sns
warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn")
np.random.seed(42)
tf.random.set_seed(42)

In [2]:
df = pd.read_csv('newww (2).csv') 

In [3]:
y = df['label']
X = df.drop('label', axis = 1)

In [4]:
def prepare_data(X, window_size = 10):
    
    data = []
    
    for i in range(len(X)):
        row = X.iloc[i].values
        row_data = []
        for j in range(len(row) - window_size):
            window = row[j : j + window_size]
            row_data.append(window)
        data.append(row_data)
        
    return np.array(data)
X.iloc[:, :] = MinMaxScaler().fit_transform(X)
new_X = prepare_data(X, window_size = 5)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(new_X, y.values, test_size=0.2, shuffle = True, stratify = y.values, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle = True, stratify = y_train, random_state=42)

In [6]:
from keras.models import Sequential
from keras.layers import Input, LSTM, Dropout, Dense
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy

model = Sequential()

input_shape = (X_train.shape[1], X_train.shape[2])
model.add(Input(shape=input_shape))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))


model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.2))


model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))


model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=BinaryCrossentropy(), metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 21, 64)              │          17,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 21, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 21, 64)              │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 21, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 21, 64)              │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 21, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 123,265 (481.50 KB)

 Trainable params: 123,265 (481.50 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

callback = [
    ModelCheckpoint(filepath='model.keras', save_best_only=True, monitor='val_loss', mode='min', verbose=1),
    EarlyStopping(monitor='val_loss', patience=10, baseline=None, restore_best_weights=True, verbose=1)
]

In [8]:
history = model.fit(X_train, y_train, batch_size=32, epochs=60, validation_data = (X_val, y_val), callbacks = callback)

Epoch 1/60
235/236 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5002 - loss: 0.6940
Epoch 1: val_loss improved from inf to 0.69283, saving model to model.keras
236/236 ━━━━━━━━━━━━━━━━━━━━ 16s 38ms/step - accuracy: 0.5003 - loss: 0.6940 - val_accuracy: 0.4997 - val_loss: 0.6928
Epoch 2/60
235/236 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.5252 - loss: 0.6866
Epoch 2: val_loss improved from 0.69283 to 0.62051, saving model to model.keras
236/236 ━━━━━━━━━━━━━━━━━━━━ 7s 31ms/step - accuracy: 0.5257 - loss: 0.6865 - val_accuracy: 0.6626 - val_loss: 0.6205
Epoch 3/60
235/236 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.6454 - loss: 0.6386
Epoch 3: val_loss improved from 0.62051 to 0.58805, saving model to model.keras
236/236 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - accuracy: 0.6455 - loss: 0.6385 - val_accuracy: 0.6976 - val_loss: 0.5881
Epoch 4/60
236/236 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.7037 - loss: 0.5848
Epoch 4: val_loss improved from 0.58805 to 0.45283, saving m

In [10]:
y_pred=model.predict(X_test)
y_pred.shape

74/74 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step


(2356, 1)

In [11]:
y_test.shape

(2356,)

In [12]:
y_test

array([1, 1, 0, ..., 0, 0, 1], dtype=int64)

In [13]:
y_pred.shape

(2356, 1)

In [14]:
y_pred

array([[9.9999869e-01],
       [1.0000000e+00],
       [1.3484157e-05],
       ...,
       [3.6338683e-05],
       [1.7358431e-03],
       [9.9999946e-01]], dtype=float32)

In [15]:
y_pred= (y_pred >= 0.5).astype(int)
f1=f1_score(y_test,y_pred)

f1


0.9843419382141346